# Model Comparison

## 1. Simulation Setup

In [ ]:
library(RandomFields)
library(fields)
library(orthopolynom)
library(mvtnorm)
library(combinat)
library(lattice)
library(Matrix)
library(limSolve)
library(MCMCpack)
library(truncnorm)
library(ggplot2)
library(tidyverse)
library(doParallel)
library(foreach)
library(spatstat)
library(inlabru)
library(sf)
library(sp)
library(INLA)
library(BART)
library(splines)
source("Functions/basket_func_same.R")
source('Functions/SVC-LGCP.R')
save_folder = 'Data/ModelComparison'
if (!dir.exists(save_folder)) {
      dir.create(save_folder)
}
simu_info = list(
    num_rep = 10,
    settings = c('jordan','curry','james'),
    num_cores = 15, 
    p_thinning = 0.8, 
    delta_lattice = 0.05,
    num_save = 5000,
    num_burn = 10000,
    update_saved = FALSE
)

simu_setting_df <- expand.grid(setting = simu_info$settings, i_rep = seq(simu_info$num_rep))
num_simu_setting = dim(simu_setting_df)[1]

## 2. Model Fitting

The fitting code below runs in parallel using `simu_info$num_cores` threads, and will take a few hours. The fitting results are already saved in their corresponding locations.

In [ ]:
cl = makeCluster(simu_info$num_cores, outfile=sprintf('%s/dopar_log.out', save_folder))
registerDoParallel(cl)

result__ = foreach(i_simu_setting = 1:num_simu_setting, 
              .packages = c("RandomFields","orthopolynom","mvtnorm","combinat","lattice","Matrix",
                           "limSolve","MCMCpack","truncnorm","spatstat","splines","inlabru","sf","sp","INLA","fields","BART","splines","ggplot2"), .verbose = TRUE, .errorhandling = "stop") %dopar% {
    source("Functions/basket_func_same.R")
    source('Functions/SVC-LGCP.R')

    # Load setting
    # -------------------------------------
    i_rep = simu_setting_df$i_rep[i_simu_setting]
    setting = simu_setting_df$setting[i_simu_setting]
    
    # Create subfolder for settings
    # -------------------------------------
    setting_string = setting
    save_subfolder = sprintf('%s/%s', save_folder, setting_string)
    if (!dir.exists(save_subfolder)) {
      dir.create(save_subfolder)
    }
 
    # Define function
    # -------------------------------------
    logintensity_function <- function(x, y, z, type, theta_alpha, theta_beta_type0, theta_beta_type1, n, a, b) {

        coords = matrix(0, length(x), 2)
        coords[,1] = x;  coords[,2] = y 
        GP_basis_value = GP.eigen.funcs(coords, n=n, a=a, b=b) 

        if(type == 0) logintensity = GP_basis_value%*%theta_alpha + GP_basis_value %*% theta_beta_type0 %*% z
        if(type == 1) logintensity = GP_basis_value%*%theta_alpha + GP_basis_value %*% theta_beta_type1 %*% z

      return( as.vector(logintensity) ) 
    }
    intensity_function <- function(x, y, z, type , theta_alpha, theta_beta_type0, theta_beta_type1, n, a, b) {
        return(exp(logintensity_function(x = x, y = y, z = z, 
                                         type =type, theta_alpha = theta_alpha, theta_beta_type0 = theta_beta_type0, 
                                         theta_beta_type1 = theta_beta_type1, n = n, a = a, b = b)))
    }
    
    # Define mesh
    # -------------------------------------
    boundary = fm_segm(rbind(c(0, 0), c(1, 0), c(1, 1), c(0, 1))*2-1, is.bnd = TRUE, crs = NA_character_)
    delta_lattice = simu_info$delta_lattice
    lattice = fm_lattice_2d(x = seq(-1,1,by = delta_lattice), y = seq(-1,1,by = delta_lattice), type = 'R', boundary = boundary)
    mesh = fm_rcdt_2d(lattice = lattice, boundary = boundary, crs = NA_character_) # ~120K vertex
    vrt = fm_vertices(mesh) # spdf for vertex coordinates
    vrt_coords = st_coordinates(vrt)[,1:2] # matrix with columns X and Y

    boundary_poly = Polygon(rbind(c(0, 0), c(1, 0), c(1, 1), c(0, 1))*2-1)
    boundary_poly = SpatialPolygons(list(Polygons(list(boundary_poly), ID = '1')))

    window = owin(c(-1,1),c(-1,1))

    
    # Load data
    # -------------------------------------
    if(setting != 'jordan'){
        covar2<-read.table(sprintf("Data/%s/covariates.txt",setting),header=TRUE)
    }else{
        covar2<-read.table(sprintf("Data/%s/covariates_home.txt",setting),header=TRUE)
    }
    
    home = away = rep(0, nrow(covar2))
    home[covar2$home==1] = 1
    away[covar2$home==2] = 1

    level = abs(covar2$level - 2)
    covar = data.frame(game = covar2$game, home, away, level = level)
    if(setting != 'jordan'){
        shot_data = read.table(sprintf("Data/%s/del_%s.txt",setting,setting),header=TRUE)
        shot_data[, 3] = shot_data[, 3]/250 
        shot_data[, 4] = shot_data[, 4]/235 - 39/47
    
    }else{
        shot_data = read.table(sprintf("Data/%s/newdata.txt",setting),header=TRUE)
        bigid = which(shot_data[,6]>28)
        smallid = which(shot_data[, 6]<1)

        shot_data = shot_data[-c(smallid,bigid), ]
        shot_data[, 4] = shot_data[, 4]/300 - 1
        shot_data[, 5] = shot_data[, 5]/235 - 1
    }
    
    # --------------------# --------------------# --------------------
    # DGP
    set.seed(i_rep)
    is_selected = sample(x = c(TRUE, FALSE), size = dim(shot_data)[1], replace = TRUE, prob = c(simu_info$p_thinning, 1 - simu_info$p_thinning))
    shot_data_ALL = shot_data
    shot_data_test = shot_data_ALL[!is_selected, ]
    shot_data = shot_data_ALL[is_selected, ]
    
    p=dim(covar)[2]-1
    courtgrid = as.matrix(expand.grid(seq(-1,1,length=50),seq(-1,1,length=40)))
    intergrid = as.matrix(expand.grid(seq(-1,1,length=60),seq(-1,1,length=60)))
    dxdy=(intergrid[1,1]-intergrid[2,1])^2
    a = 0.25
    d = 2
    b = 1.5
    #GP.eigen.degree(0.9,a,b=9,d=2)
    GP.eigen.degree(0.8,a=a,b=b,d=d)
    n = GP.eigen.degree(0.8,a=a,b=b,d=d)$n
    eigmat = GP.eigen.value(n=n,a=a,b=b,d=d)
    eigmat = diag(eigmat)
    pmat = eigmat
    L = nrow(eigmat)

    intermat = GP.eigen.funcs(intergrid, n=n, a=a, b=b)
    courtmat = GP.eigen.funcs(courtgrid, n=n, a=a, b=b)

    num.type = 2
    apop = 5
    bpop = 5
    sigma.pop = 1#rinvgamma(1,apop,bpop)
    atype = 5
    btype = 5
    sigma.type = rep(1,num.type*p)#rinvgamma(num.type*p,atype,btype)
    N = simu_info$num_burn + simu_info$num_save    
    # --------------------# --------------------# --------------------
    # Prepar training data
    shot_data_ALL = cbind(shot_data_ALL, covar[shot_data_ALL$Game,c('home','away','level')])
    shot_coords_ALL = as.matrix(shot_data_ALL[,c('Xdist','Ydist')])

    shot_data_sp_ALL = SpatialPoints(coords = shot_coords_ALL, bbox = as.matrix(data.frame(min = c(-1,-1), max = c(1,1))))
    shot_data_spdf_ALL = SpatialPointsDataFrame(coords = shot_coords_ALL, 
                                            data = cbind(shot_data_ALL$Missmade, covar[shot_data_ALL$Game,c('home','away','level')]), 
                                            bbox = as.matrix(data.frame(min = c(-1,-1), max = c(1,1))), match.ID = FALSE)

    shot_data_ppp_ALL = ppp(x = shot_data_ALL$Xdist, y = shot_data_ALL$Ydist, window = window )
        
    num_data_ALL = dim(shot_coords_ALL)[1]

    shot_data = shot_data_ALL[is_selected, ]
    shot_coords = shot_coords_ALL[is_selected, ]
    shot_data_sp = shot_data_sp_ALL[is_selected, ]
    shot_data_spdf = shot_data_spdf_ALL[is_selected, ]
    shot_data_ppp = shot_data_ppp_ALL[is_selected, ]
    num_data = dim(shot_coords)[1]
    
    shot_data_test = shot_data_ALL[!is_selected, ]
    shot_coords_test = shot_coords_ALL[!is_selected, ]
    shot_data_sp_test = shot_data_sp_ALL[!is_selected, ]
    shot_data_spdf_test = shot_data_spdf_ALL[!is_selected, ]
    shot_data_ppp_test = shot_data_ppp_ALL[!is_selected, ]
    num_data_test = dim(shot_coords_test)[1]
    
    if(i_rep == 1){
        figure = ggplot(data = shot_data) + geom_point(aes(x = Xdist, y = Ydist, color = Missmade)) + facet_grid(rows = vars(home), cols = vars(level))
        ggsave(figure, file = sprintf('%s/Points_rep=%d.pdf', save_subfolder, i_rep))
    }

    # --------------------# --------------------# --------------------
    # Get grid data for MAE
    lattice_coords = as.matrix(expand.grid( x = seq(simu_info$delta_lattice/2 - 1, 1, by = simu_info$delta_lattice), y = seq(simu_info$delta_lattice/2 - 1 , 1, by = simu_info$delta_lattice)  ) )
    num_lattice_blocks = dim(lattice_coords)[1]
    
    x.grid = matrix(0, 1, 2 + 3)
    shot_count = c()
    shot_count_test = c()
    for(home in c(0,1)){for(level in c(0,1)){for(type in c(0,1)){ 
        num_game_home_level = sum( (covar[, 'home'] == home) & (covar[, 'level'] == level) )

        id_home_level_type = which((covar[shot_data$Game, 'home'] == home) & (covar[shot_data$Game, 'level'] == level) & (shot_data$Missmade == type) )
        shot_coords_home_level_type = shot_coords[id_home_level_type, ]
        i_x_grid = ceiling( (shot_coords_home_level_type[,1] + 1) / simu_info$delta_lattice)
        i_y_grid = ceiling( (shot_coords_home_level_type[,2] + 1) / simu_info$delta_lattice)
        i_grid = i_x_grid + (2 / simu_info$delta_lattice) * (i_y_grid - 1)
        table_count_home_level_type = table(i_grid)
        shot_count_home_level_type = rep(0, num_lattice_blocks)
        shot_count_home_level_type[as.numeric(names(table_count_home_level_type))] = as.numeric(table_count_home_level_type)
        shot_count = c(shot_count, shot_count_home_level_type)

        id_home_level_type = which((covar[shot_data_test$Game, 'home'] == home) & (covar[shot_data_test$Game, 'level'] == level) & (shot_data_test$Missmade == type) )
        shot_coords_home_level_type = shot_coords_test[id_home_level_type, ]
        i_x_grid = ceiling( (shot_coords_home_level_type[,1] + 1) / simu_info$delta_lattice)
        i_y_grid = ceiling( (shot_coords_home_level_type[,2] + 1) / simu_info$delta_lattice)
        i_grid = i_x_grid + (2 / simu_info$delta_lattice) * (i_y_grid - 1)
        table_count_home_level_type = table(i_grid)
        shot_count_home_level_type = rep(0, num_lattice_blocks)
        shot_count_home_level_type[as.numeric(names(table_count_home_level_type))] = as.numeric(table_count_home_level_type)
        shot_count_test = c(shot_count_test, shot_count_home_level_type)
                                           
        x.grid = rbind(x.grid, cbind(lattice_coords, matrix(c(home, level, type), nrow = dim(lattice_coords)[1], ncol = 3, byrow = TRUE) ))
    }}}                                 
    x.grid = x.grid[-1, ]
    shot_grid_ppp = ppp(x = x.grid[,1], y = x.grid[,2], window = window )
    shot_grid_spdf = SpatialPointsDataFrame(coords = x.grid[,1:2], 
                                            data = as.data.frame(x.grid[,3:5]), 
                                            bbox = as.matrix(data.frame(min = c(-1,-1), max = c(1,1))), match.ID = FALSE)
    num_data_lattice = dim(x.grid)[1]

    
    # --------------------# --------------------# --------------------
    # 2. Fitting
    # --------------------# --------------------# --------------------
    # 2.1 SVC_LGCP
    # --------------------
    save_file_name = sprintf('%s/SVCLGCP_rep=%d.RData', save_subfolder, i_rep)
    if(!file.exists(save_file_name) | (simu_info$update_saved == TRUE) ){

        num.type = 2
        apop = 5
        bpop = 5
        sigma.pop = 1#rinvgamma(1,apop,bpop)
        atype = 5
        btype = 5
        sigma.type = rep(1,num.type*p)#rinvgamma(num.type*p,atype,btype)
        N = simu_info$num_burn + simu_info$num_save
        fit = fit_SVCLGCP(shot_data, covar) 
        
        ##: list(Theta.result = Theta.result, likelihood1 = likelihood1, Sigmapop = Sigmapop, Sigmatype = Sigmatype)
        ##: fit$Theta.result: [isim, i_L, j_type]
        
        intensity_pred = rep(0 , num_data)
        intensity_pred_test = rep(0 , num_data_test)
        intensity_pred_grid = rep(0 , num_data_lattice)
        theta_hat = apply(fit$Theta.result[simu_info$num_burn+1:simu_info$num_save,,],c(2,3),mean)
        for(home in c(0,1)){for(level in c(0,1)){for(type in c(0,1)){            
            id_home_level_type = which((covar[shot_data$Game, 'home'] == home) & (covar[shot_data$Game, 'level'] == level) & (shot_data$Missmade == type) )
            if(length(id_home_level_type)>0){
                intensity_pred[id_home_level_type] = intensity_function(shot_coords[id_home_level_type, 1],  shot_coords[id_home_level_type, 2],
                                                c(home, 1 - home, level),
                                                type = type,
                                                theta_alpha = theta_hat[,1],
                                                theta_beta_type0 = theta_hat[,2:(p+1)],
                                                theta_beta_type1 = theta_hat[,(p+2):(2*p+1)],
                                                n = n, a = a, b = b)
             }
            
            id_home_level_type_test = which((covar[shot_data_test$Game, 'home'] == home) & (covar[shot_data_test$Game, 'level'] == level) & (shot_data_test$Missmade == type) )
            if(length(id_home_level_type_test)>0){
                intensity_pred_test[id_home_level_type_test] = intensity_function(shot_coords_test[id_home_level_type_test, 1],  shot_coords_test[id_home_level_type_test, 2],
                                                c(home, 1 - home, level),
                                                type = type,
                                                theta_alpha = theta_hat[,1],
                                                theta_beta_type0 = theta_hat[,2:(p+1)],
                                                theta_beta_type1 = theta_hat[,(p+2):(2*p+1)],
                                                n = n, a = a, b = b)
             }
            
            id_home_level_type_grid = which((x.grid[,3] == home) & (x.grid[,4] == level) & (x.grid[,5] == type) )
            if(length(id_home_level_type_grid)>0){
                intensity_pred_grid[id_home_level_type_grid] = intensity_function(x.grid[id_home_level_type_grid, 2],  x.grid[id_home_level_type_grid, 3],
                                                c(home, 1 - home, level),
                                                type = type,
                                                theta_alpha = theta_hat[,1],
                                                theta_beta_type0 = theta_hat[,2:(p+1)],
                                                theta_beta_type1 = theta_hat[,(p+2):(2*p+1)],
                                                n = n, a = a, b = b)
             }
    
        }}}

        MAE = mean( abs( (1-simu_info$p_thinning)/simu_info$p_thinning * intensity_pred_grid * (simu_info$delta_lattice)^2 - shot_count_test  ) )
        npllg = -sum(dpois(shot_count_test, intensity_pred_test * (simu_info$delta_lattice)^2 , log = TRUE))
        save(fit, intensity_pred, intensity_pred_test, MAE, npllg, file = save_file_name)
    }
    
   
    # 2.2 LGCP
    # --------------------
    save_file_name = sprintf('%s/LGCP_rep=%d.RData', save_subfolder, i_rep)
    if(!file.exists(save_file_name) | (simu_info$update_saved == TRUE) ){
    
        intensity_pred = rep(0 , num_data)
        intensity_pred_test = rep(0 , num_data_test)
        intensity_pred_grid = rep(0 , num_data_lattice)
        for(home in c(0,1)){for(level in c(0,1)){for(type in c(0,1)){ 
            num_game_home_level = sum( (covar[, 'home'] == home) & (covar[, 'level'] == level) )
            
            id_home_level_type = which((covar[shot_data$Game, 'home'] == home) & (covar[shot_data$Game, 'level'] == level) & (shot_data$Missmade == type) )
            
            if(length(id_home_level_type)>0){
                matern = inla.spde2.pcmatern(mesh,
                        prior.sigma = c(5, 0.1),
                        prior.range = c(2, 0.1))
                fit = lgcp(components = coordinates ~ Intercept(1) + mySPDE(map = coordinates, model = matern), 
                            data = shot_data_spdf[id_home_level_type,],
                            domain = list(coordinates = mesh),
                            samplers = boundary_poly)
                intensity_pred[ id_home_level_type ] = predict(fit, shot_data_spdf[id_home_level_type,], ~ exp(mySPDE + Intercept))$mean / num_game_home_level
                
                id_home_level_type_test = which((covar[shot_data_test$Game, 'home'] == home) & (covar[shot_data_test$Game, 'level'] == level) & (shot_data_test$Missmade == type) )
                intensity_pred_test[ id_home_level_type_test ] = predict(fit, shot_data_spdf_test[id_home_level_type_test,], ~ exp(mySPDE + Intercept))$mean / num_game_home_level
                
                id_home_level_type_gird = which((x.grid[,3] == home) & (x.grid[,4] == level) & (x.grid[,5] == type) )
                intensity_pred_grid[ id_home_level_type_gird ] = predict(fit, shot_grid_spdf[id_home_level_type_gird,], ~ exp(mySPDE + Intercept))$mean / num_game_home_level
            }
        }}}

        MAE = mean( abs( (1-simu_info$p_thinning)/simu_info$p_thinning * intensity_pred_grid * (simu_info$delta_lattice)^2 - shot_count_test  ) )
        npllg = -sum(dpois(shot_count_test, intensity_pred_test * (simu_info$delta_lattice)^2, log = TRUE))
        save(intensity_pred, intensity_pred_test, MAE, npllg, file = save_file_name)
    }

    # 2.3 Inhomogeneous Poisson process
    # --------------------
    save_file_name = sprintf('%s/IPP_rep=%d.RData', save_subfolder, i_rep)
    if(!file.exists(save_file_name) | (simu_info$update_saved == TRUE) ){

        fit_list = vector(mode = 'list', length = 8)
        intensity_pred = rep(0 , num_data)
        intensity_pred_test = rep(0 , num_data_test)
        intensity_pred_grid = rep(0 , num_data_lattice)
        counter = 0
        for(home in c(0,1)){for(level in c(0,1)){for(type in c(0,1)){ 
            counter = counter + 1
            num_game_home_level = sum( (covar[, 'home'] == home) & (covar[, 'level'] == level) )
            
            id_home_level_type = which((covar[shot_data$Game, 'home'] == home) & (covar[shot_data$Game, 'level'] == level) & (shot_data$Missmade == type) )
            
            if(length(id_home_level_type)>0){
                fit = ppm(shot_data_ppp[id_home_level_type, ], ~ bs(x, 6) + bs(y, 6))
                fit_list[[counter]] = fit
                intensity_pred[ id_home_level_type ] = predict(fit, locations = shot_data_ppp[id_home_level_type,]) / num_game_home_level
                
                id_home_level_type_test = which((covar[shot_data_test$Game, 'home'] == home) & (covar[shot_data_test$Game, 'level'] == level) & (shot_data_test$Missmade == type) )
                intensity_pred_test[ id_home_level_type_test ] = predict(fit, locations = shot_data_ppp_test[id_home_level_type_test,]) / num_game_home_level
                
                id_home_level_type_gird = which((x.grid[,3] == home) & (x.grid[,4] == level) & (x.grid[,5] == type) )
                intensity_pred_grid[ id_home_level_type_gird ] = predict(fit, locations = shot_grid_ppp[id_home_level_type_gird, ]) / num_game_home_level
                
            }
        }}}

        MAE = mean( abs( (1-simu_info$p_thinning)/simu_info$p_thinning * intensity_pred_grid * (simu_info$delta_lattice)^2 - shot_count_test  ) )
        npllg = -sum(dpois(shot_count_test, intensity_pred_test * (simu_info$delta_lattice)^2, log = TRUE))
        save(fit_list, intensity_pred, intensity_pred_test, MAE, npllg, file = save_file_name)
    }
    
    # 2.4 KDE
    # --------------------
    save_file_name = sprintf('%s/KDE_rep=%d.RData', save_subfolder, i_rep)
    if(!file.exists(save_file_name) | (simu_info$update_saved == TRUE) ){
        sigma_kde_list = c(seq(0.05, 1, by = 0.1), seq(1.2,2,by = 0.2))
        
        MAE_best = Inf
        sigma_kde_best = 0
        
        for(sigma_kde in sigma_kde_list){
            intensity_pred = rep(0 , num_data)
            intensity_pred_test = rep(0 , num_data_test)
            intensity_pred_grid = rep(0 , num_data_lattice)
            for(home in c(0,1)){for(level in c(0,1)){for(type in c(0,1)){ 
                num_game_home_level = sum( (covar[, 'home'] == home) & (covar[, 'level'] == level) )

                id_home_level_type = which((covar[shot_data$Game, 'home'] == home) & (covar[shot_data$Game, 'level'] == level) & (shot_data$Missmade == type) )
                if(length(id_home_level_type)>0){
                    fit = density.ppp(ppp(shot_coords[id_home_level_type,1], shot_coords[id_home_level_type,2], c(-1,1),c(-1,1)), sigma = sigma_kde)
                    intensity_pred[id_home_level_type] = interp.im(fit, x = shot_coords[id_home_level_type,1], y = shot_coords[id_home_level_type,2])/num_game_home_level
                    
                    id_home_level_type_test = which((covar[shot_data_test$Game, 'home'] == home) & (covar[shot_data_test$Game, 'level'] == level) & (shot_data_test$Missmade == type) )
                    intensity_pred_test[ id_home_level_type_test ] = interp.im(fit, x = shot_coords_test[id_home_level_type_test,1], y = shot_coords_test[id_home_level_type_test,2])/num_game_home_level

                    id_home_level_type_gird = which((x.grid[,3] == home) & (x.grid[,4] == level) & (x.grid[,5] == type) )
                    intensity_pred_grid[ id_home_level_type_gird ] = interp.im(fit, x = x.grid[id_home_level_type_gird,1], y = x.grid[id_home_level_type_gird,2])/num_game_home_level
                }
            }}}
            MAE = mean( abs( (1-simu_info$p_thinning)/simu_info$p_thinning * intensity_pred_grid * (simu_info$delta_lattice)^2 - shot_count_test  ) )

            if(MAE < MAE_best){
                MAE_best = MAE
                sigma_kde_best = sigma_kde
                intensity_pred_best = intensity_pred
                intensity_test_best = intensity_pred_test
            }
        }

        npllg = -sum(dpois(shot_count_test, intensity_pred_test * (simu_info$delta_lattice)^2, log = TRUE))
        save(sigma_kde_best, intensity_pred_best, intensity_test_best, MAE_best, npllg, file = save_file_name)
    }


    # 2.5 BART
    # --------------------
    save_file_name = sprintf('%s/BART_rep=%d.RData', save_subfolder, i_rep)
    if(!file.exists(save_file_name) | (simu_info$update_saved == TRUE) ){
        shot_count_compress = shot_count
        shot_count_compress[shot_count_compress > 1] = 1        
        fit = gbart(x.train = x.grid, y.train = shot_count_compress, ntree = 50, type = 'pbart', nskip = simu_info$num_burn, ndpost = simu_info$num_save)
        intensity_pred = predict( fit, newdata = cbind(shot_coords, shot_data_spdf@data[,-3]) )$prob.test.mean / (simu_info$delta_lattice^2) / num_game_home_level

        intensity_pred_test = predict( fit, newdata = cbind(shot_coords_test, shot_data_spdf_test@data[,-3]) )$prob.test.mean / (simu_info$delta_lattice^2) / num_game_home_level
        
        intensity_pred_grid = predict( fit, newdata = x.grid )$prob.test.mean / (simu_info$delta_lattice^2) / num_game_home_level
        
        MAE = mean( abs( (1-simu_info$p_thinning)/simu_info$p_thinning * intensity_pred_grid * (simu_info$delta_lattice)^2 - shot_count_test  ) )
        npllg = -sum(dpois(shot_count_test, intensity_pred_test * (simu_info$delta_lattice)^2, log = TRUE))
        save(intensity_pred, intensity_pred_test, MAE, npllg, file = save_file_name)                   
    }

}

stopCluster(cl)                                            

## 3. Fitting Results (Figure 9)

In [4]:
library(readxl)
library(devtools)
library(dplyr)
library(ggbasketball)
library(combinat)
library(lattice)
library(gridExtra)
library(grid) 
library(patchwork)
library(orthopolynom)
simu_info = list(
    num_rep = 10,
    settings = c('jordan','curry','james'),
    num_cores = 15, 
    p_thinning = 0.8, 
    delta_lattice = 0.05,
    num_save = 5000,
    num_burn = 10000,
    update_saved = FALSE
)
save_folder = 'Data/ModelComparison'

simu_setting_df <- expand.grid(setting = simu_info$settings, i_rep = seq(simu_info$num_rep))
num_simu_setting = dim(simu_setting_df)[1]

df_result_SVCLGCP = df_result_LGCP = df_result_IPP = df_result_KDE= df_result_BART = simu_setting_df

for(i_simu_setting in 1:num_simu_setting){

    # Load setting
    # -------------------------------------
    i_rep = simu_setting_df$i_rep[i_simu_setting]
    setting = simu_setting_df$setting[i_simu_setting]
    setting_string = setting
    save_subfolder = sprintf('%s/%s', save_folder, setting_string)
    
    # 2.1 SVC_LGCP
    # --------------------
    save_file_name = sprintf('%s/SVCLGCP_rep=%d.RData', save_subfolder, i_rep)
    load(file = save_file_name)
    df_result_SVCLGCP$NPLL[i_simu_setting] = npllg

    # 2.2 LGCP
    # --------------------
    save_file_name = sprintf('%s/LGCP_rep=%d.RData', save_subfolder, i_rep)
    load(file = save_file_name)
    df_result_LGCP$NPLL[i_simu_setting] = npllg
    
    # 2.3 IPP
    # --------------------
    save_file_name = sprintf('%s/IPP_rep=%d.RData', save_subfolder, i_rep)
    load(file = save_file_name)
    df_result_IPP$NPLL[i_simu_setting] = npllg
    
    # 2.4 KDE
    # --------------------
    save_file_name = sprintf('%s/KDE_rep=%d.RData', save_subfolder, i_rep)
    load(file = save_file_name)
    df_result_KDE$NPLL[i_simu_setting] = npllg
    
    # 2.5 BART
    # --------------------
    save_file_name = sprintf('%s/BART_rep=%d.RData', save_subfolder, i_rep)
    load(file = save_file_name)
    df_result_BART$NPLL[i_simu_setting] = npllg
}

In [5]:
df_result = rbind(df_result_SVCLGCP, df_result_LGCP, df_result_IPP, df_result_KDE, df_result_BART)
methd_list = c('JVCLGCP','LGCP','IPP','KDE','BART')
df_result$method = factor(rep(methd_list, each = dim(df_result_SVCLGCP)[1]), levels = methd_list)
df = df_result %>% pivot_longer(
    cols = -c(i_rep, setting, method), names_to = "metric", values_to = "value") %>% 
       group_by(setting, method)
df$setting = factor(df$setting, levels = c('curry','james','jordan'))
levels(df$setting) = c('Stephen Curry', 'LeBron James','Michael Jordan')

p_boxplot = ggplot(df %>% filter(metric == 'NPLL')) +
      geom_boxplot(aes(x = method, y = value)) +
      labs(x = NULL, y = 'NPLL') +
      facet_grid(cols = vars(setting)) +
      theme(
        legend.key.size = unit(1, 'cm'),
        legend.title = element_text(size = 20),
        legend.text = element_text(size = 25),
        axis.text.x = element_text(angle = 45, hjust = 1, size = 25),  # Rotate x-axis text
        axis.text.y = element_text(size = 25),  # Adjusted size for better readability
        axis.title.x = element_text(size = 25),  # Adjusted size for better readability
        axis.title.y = element_text(size = 25, margin = margin(r = 30)),  
        strip.text = element_text(size = 30),  # Adjusted size for better readability
        panel.spacing = unit(1, 'lines')
      ) +
      scale_alpha_manual(values = c(1, 0.5))

ggsave(p_boxplot ,filename = sprintf('%s/boxplot.pdf', save_folder), width = 12, height = 7)